In [1]:
import sqlite3
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy import stats
import mercury as mr

In [2]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    mr.Markdown(f"## Es liegen noch keine Daten vor.")

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_jp.db")
cur = con.cursor()

In [3]:
def DB_Select_Alle_MeSH():
    cur.execute("SELECT Term FROM MeSH")
    data = cur.fetchall()
    term = [x[0] for x in data]
    return term

def DB_Select_Alle_Dokumente_MeSH(meshUI):
    cur.execute("SELECT Titel, Dokument.PMID FROM Dokument JOIN MeSH_Dok ON Dokument.PMID = MeSH_Dok.PMID WHERE UI = '" + str(meshUI) +"'")
    output = cur.fetchall()
    titles = [x[0] for x in output]
    json_list={}
    for title, pmid in output:
        json_list[title] = pmid
    return json_list, titles

def DB_Select_Term(UI):
    cur.execute("SELECT Term FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_MeSH_UI(term):
    cur.execute("SELECT UI FROM MeSH WHERE Term = '" + str(term) + "'")
    MeSH_UI =  cur.fetchall()[0][0]
    return MeSH_UI

def sortDataframes(dataframe):
    return len(dataframe.index)

In [4]:
# Funktion zur Visualsierung der kumulierten Zitationszahlen
# Ausgabe von Eckdaten

def visualisiereGesamtUeberblick():
    try:
        cur.execute("SELECT count(UI) FROM MeSH")
        count_mesh = cur.fetchall()[0][0]
        x = ["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"]
        fig = plt.figure(figsize=(15, 10))
        sub1 = plt.subplot(2, 2, 1)
        sub2 = plt.subplot(2, 2, 2)
        # Daten Plot 1
        cur.execute("SELECT sum(EK), sum(NEK), sum(NEKK) FROM MeSH")
        data = cur.fetchall()[0]
        y1 = {"gesamt": [data[0], data[1], data[2]]}
        df1=pd.DataFrame(y1,index=x)
        maxZitation = max(y1["gesamt"])
        # Daten Plot 2
        cur.execute("SELECT EK, NEK, NEKK, UI FROM MeSH")
        data = cur.fetchall()
        y2 = {}
        for ek, nek, nekk, ui in data:
            y2[ui] = [ek, nek, nekk]
        df2=pd.DataFrame(y2,index=x)
        # Plot 
        df1.plot(ax=sub1,kind="bar",stacked=True,legend=None)
        df2.plot(ax=sub2,kind="bar",stacked=True,legend=None)
        if len(y2) > 15:
            columns = math.ceil(len(y2)/15)
            sub2.legend(loc=(1.01, 0.01), title="MeSH UI", ncols=columns)
        else:
            sub2.legend(loc="upper right", bbox_to_anchor=(1.3, 1), title="Dokument PMID")
        
        for i in range(len(x)):
            sub1.text(i, y1["gesamt"][i]+0.015*maxZitation, y1["gesamt"][i], ha = 'center')
            sub2.text(i, y1["gesamt"][i]+0.015*maxZitation, y1["gesamt"][i], ha = 'center')
        sub1.set_title("Gesamtüberblick")
        sub2.set_title("Anzahl analystierter MeSH: " + str(count_mesh))
        sub1.tick_params("x", labelrotation=45)
        sub1.set_ylabel("Anzahl Zitationen")
        sub2.tick_params("x", labelrotation=45)
        sub2.set_ylabel("Anzahl Zitationen")
        plt.plot()
    except:
        mr.Markdown(f"## Keine Daten vorhanden")

def visualisiereMeSHZeitstrahl(meshUI):
    try:
        fig = plt.figure(figsize=(15, 10))
        sub1 = plt.subplot(2, 2, 1)
        sub2 = plt.subplot(2, 2, 2)
        cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(meshUI) + "' ")
        mesh_jahr = cur.fetchall()[0][0]
        # Plot 1
        cur.execute("SELECT Jahr, sum(Anzahl) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' GROUP BY UI,Jahr")
        data = cur.fetchall()
        x1 = [dat[0] for dat in data]
        y1 = [dat[1] for dat in data]
        df1 = pd.DataFrame(y1,index=x1)
        df1.plot(ax=sub1,kind="bar",stacked=True,legend=None)
        sub1.tick_params("x", labelrotation=45)
        sub1.set_title("Anzahl der Zitationen für das MeSH: " + str(meshUI) + " \n Aufnahmejahr: " + str(mesh_jahr))
        sub1.set_xlabel("Jahr")
        sub1.set_ylabel("Anzahl Zitationen")
        # Plot 2
        cur.execute("SELECT DISTINCT(Jahr) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' ORDER BY Jahr")    
        jahre = [x[0] for x in cur.fetchall()]
        cur.execute("SELECT DISTINCT(Zitationen.PMID) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' ")    
        pmids = [x[0] for x in cur.fetchall()]
        data = {}
        for pmid in pmids:
            data[pmid] = np.zeros(len(jahre))
        df2 = pd.DataFrame(data, index=jahre)
        cur.execute("SELECT Zitationen.PMID, Jahr, Anzahl FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "'  ORDER BY Jahr")        
        data = cur.fetchall()
        for pmid, jahr, anzahl in data:
            df2.loc[jahr, pmid] = anzahl
        df2.plot(ax=sub2,kind="bar",stacked=True,legend=None)
        sub2.tick_params("x", labelrotation=45)
        if len(pmids) > 15:
            columns = math.ceil(len(pmids)/18)
            sub2.legend(loc=(1.01, 0.01), title="Dokument PMID", ncols=columns)
        else:
            sub2.legend(loc="upper right", bbox_to_anchor=(1.3, 1), title="Dokument PMID")
        sub2.set_title("Anzahl der Zitationen für das MeSH: " + str(meshUI) + " pro Dokument\n Anzahl analysierter Dokumente: "+ str(len(pmids)))
        sub2.set_xlabel("Jahr")
        sub2.set_ylabel("Anzahl Zitationen")
        if len(jahre) > 15:
            for label in sub1.xaxis.get_ticklabels()[::2]:
                label.set_visible(False)
            for label in sub2.xaxis.get_ticklabels()[::2]:
                label.set_visible(False)
    except:
        print("Keine Daten verfügbar")


def visualisiereKorrelationen():
    # Plot 1: Je mehr emergente Zitate, desto mehr Zitate insgesamt
    # Merkmal X
    cur.execute("SELECT EK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    X = [x[0] for x in cur.fetchall()] # Einzelne emergente Zitationszahlen je Dokument

    # Merkmal Y
    cur.execute("SELECT EK_Dok+NEK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    Y = [y[0] for y in cur.fetchall()] # Einzelne gesamte Zitationszahlen je Dokument

    ergebnis = stats.pearsonr(X,Y)
    cor = ergebnis[0]
    # p_wert = ergebnis[1]
    mr.Markdown(f"### Korrelationskoeffizienten")
    if cor > 0: 
        mr.Markdown(f"#### Es liegt ein positiver linearer Zusammenhang zwischen emergenten und der Gesamtanzahl an Zitationen vor. Korrelationskoeffizient: {cor}")
    if cor == 0: 
        mr.Markdown(f"#### Es liegt ein neutraler linearer Zusammenhang zwischen emergenten und der Gesamtanzahl an Zitationen vor. Korrelationskoeffizient: 0")
    if cor < 0: 
        mr.Markdown(f"#### Es liegt ein negativer linearer Zusammenhang zwischen emergenten und der Gesamtanzahl an Zitationen vor. Korrelationskoeffizient: {cor}")
    # if p_wert < 0.05:
    #     print("Der p-Wert ist kleiner als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation statistisch signifikant. p-Wert: ", p_wert)
    # if p_wert > 0.05:
    #     print("Der p-Wert ist größer als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation nicht statistisch signifikant. p-Wert: ", p_wert)
    # Plot 2: Wenn MeSH aufgenommen: dann Anstieg von Zitationszahlen, Verwendung korrigierter nicht-emergenter Zeitraum
    # Merkmal X
    cur.execute("SELECT EK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    X = [x[0] for x in cur.fetchall()] # Einzelne emergente Zitationszahlen je Dokument

    # Merkmal Y
    cur.execute("SELECT NEKK_Dok FROM MeSH_Dok WHERE EK_Dok IS NOT NULL")
    Y = [y[0] for y in cur.fetchall()] # Einzelne korrigiert nicht-emergente Zitationszahlen je Dokument

    ergebnis = stats.pearsonr(X,Y)
    cor = ergebnis[0]
    # p_wert = ergebnis[1]
    if cor > 0: 
        mr.Markdown(f"#### Es liegt ein positiver linearer Zusammenhang zwischen emergenten und korrigierten nicht-emergenten Zitationen vor. Korrelationskoeffizient: {cor}")
    if cor == 0: 
        mr.Markdown(f"#### Es liegt ein neutraler linearer Zusammenhang zwischen emergenten und korrigierten nicht-emergenten Zitationen vor. Korrelationskoeffizient: 0")
    if cor < 0: 
        mr.Markdown(f"#### Es liegt ein negativer linearer Zusammenhang zwischen emergenten und korrigierten nicht-emergenten Zitationen vor. Korrelationskoeffizient: {cor}")
    # if p_wert < 0.05:
    #     print("Der p-Wert ist kleiner als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation statistisch signifikant. p-Wert: ", p_wert)
    # if p_wert > 0.05:
    #     print("Der p-Wert ist größer als das gewählte Signifikanzniveau von 0,05. Damit ist die Korrelation nicht statistisch signifikant. p-Wert: ", p_wert)
    

def visualisiereMeSHAnstieg():
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok < NEKK_DOK")
    mehr = cur.fetchall()[0][0]
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok > NEKK_DOK")
    weniger = cur.fetchall()[0][0]
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok = NEKK_DOK")
    gleich = cur.fetchall()[0][0]
    cur.execute("SELECT count(PMID) FROM MeSH_Dok WHERE EK_Dok = NEKK_DOK and EK_Dok = 0 and NEK_Dok = 0 and NEKK_Dok = 0")
    null = cur.fetchall()[0][0]
    mr.Markdown(f"### Dokumentenanzahl absoluter Zitationszahlenvergleich")
    mr.Markdown(f"#### Bei " + str(mehr) + " Dokumenten steigt die Anzahl der Zitationen nach der Einführung des MeSH an. (Betrachtung des korrigierten Zeitraums)")
    mr.Markdown(f"#### Bei " + str(weniger) + " Dokumenten nimmt die Anzahl der Zitationen nach der Einführung des MeSH ab. (Betrachtung des korrigierten Zeitraums)")
    mr.Markdown(f"#### Bei " + str(gleich) + " Dokumenten bleibt die Anzahl der Zitationen nach der Einführung des MeSH gleich. Davon werden " + str(null) + " Dokumente nie zitiert.")


def visualisiereRelativeVerlaufUndMeSH(UI):
    try:
        cur.execute("SELECT UI FROM MeSH")
        meshs = [x[0] for x in cur.fetchall()]
        max_anzahl_jahre = 0
        dataframes = [] 
        for meshUI in meshs:
            data_plot = {}
            cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(meshUI) + "' ")
            mesh_jahr = cur.fetchall()[0][0]
            cur.execute("SELECT min(Jahr) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "'")
            min_jahr = cur.fetchall()[0][0]   
            if min_jahr == None:
                continue
            jahr_korrektur = mesh_jahr+mesh_jahr-min_jahr

            anzahl_jahre = mesh_jahr-min_jahr
            if anzahl_jahre > max_anzahl_jahre:
                max_anzahl_jahre = anzahl_jahre
            if anzahl_jahre <= 0:
                continue
            
            data_plot[meshUI] = np.zeros(2*(mesh_jahr-min_jahr)+1)
            jahre = np.linspace(min_jahr, jahr_korrektur, 2*(mesh_jahr-min_jahr)+1,dtype='int')
            df1 = pd.DataFrame(data_plot, index=jahre)
            df1.loc[:, meshUI] = np.zeros(2*(mesh_jahr-min_jahr)+1)
            cur.execute("SELECT Jahr, sum(Anzahl) FROM MeSH_Dok JOIN Zitationen ON MeSH_Dok.PMID = Zitationen.PMID WHERE UI = '" + str(meshUI) + "' and Jahr <= "+str(jahr_korrektur)+" GROUP BY UI,Jahr")
            data = cur.fetchall()
            x1 = [dat[0] for dat in data]
            y1 = [dat[1] for dat in data]
            for dat in data:
                df1.loc[dat[0]][meshUI] = dat[1]/sum(y1)
            index_list_jahre = df1.index.tolist()
            for index in index_list_jahre:
                df1.rename(index={index:index-index_list_jahre[math.floor(len(index_list_jahre)/2)]},inplace=True)
            dataframes.append(df1)

        # Liste sortieren nach ANzahl der index
        # print(dataframes)
        dataframes.sort(reverse=True, key=sortDataframes)
        dataframe = dataframes[0].join(dataframes[1:])
        
        fig = plt.figure(figsize=(15, 10))
        sub1 = plt.subplot(2, 2, 1)
        sub2 = plt.subplot(2, 2, 2)
        dataframe.plot(ax=sub2)
        sub2.set_title("Relative Zitationen, vor und nach MeSH-Aufnahme")
        sub2.set_xlabel("Jahre vor (-) und nach (+) MeSH-Aufnahme") 
        sub2.set_ylabel("Relative Anzahl Zitationen\n bezogen auf die Gesamtzahl im korrigierten Zeitraum") 
        if len(dataframes) > 15:
            columns = math.ceil(len(dataframes)/15)
            sub2.legend(loc=(1.01, 0.01), title="MeSH UI", ncols=columns)
        else:
            sub2.legend(loc="upper right", bbox_to_anchor=(1.3, 1), title="MeSH UI")
    except:
        print("Keine Darstellung des relativen Verlaufs möglich.")

    # Plot 2
    try:
        Term = DB_Select_Term(UI)        
        x = ["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"]
        cur.execute("SELECT EK, NEK, NEKK FROM MeSH WHERE UI = '"+ str(UI) + "'")
        data = cur.fetchall()[0]
        y1 = {"ui": [data[0], data[1], data[2]]}
        df1=pd.DataFrame(y1,index=x)
        maxZitation = max(y1["ui"])
        df1.plot(ax=sub1,kind="bar",stacked=True,legend=None)        
        sub1.tick_params("x", labelrotation=45)
        sub1.set_ylabel("Anzahl Zitationen")
        sub1.set_title("MeshUI: " + str(UI) + ", " + str(Term) + "\n Gesamtanzahl Zitationen: " + str(sum(y1["ui"])))
        for i in range(len(x)):
            sub1.text(i, y1["ui"][i]+0.015*maxZitation, y1["ui"][i], ha = 'center')
        
    except:
        print("Keine Daten vorhanden")


def infosAllgemein():
    mr.Markdown(f"### Allgemeine Kennzahlen")
    # Anzahl MeSH in DB
    cur.execute("SELECT count(UI) FROM MeSH")
    anzahlMeSH = cur.fetchall()[0][0]
    mr.Markdown(f"#### Anzahl analysierte MeSH: {anzahlMeSH}")
    # Anzahl Dokumente in DB
    cur.execute("SELECT count(PMID) FROM Dokument")
    anzahlDok = cur.fetchall()[0][0]
    mr.Markdown(f"#### Anzahl analysierte Dokumente: {anzahlDok}")
    # Dokument mit höchster Zitationszahl in DB
    cur.execute("Select PMID, max(summe) FROM (SELECT PMID, sum(Anzahl) as summe FROM Zitationen GROUP BY PMID)")
    data = cur.fetchall()[0]
    pmid_max = data[0]
    anzahl_max = data[1]
    cur.execute("Select Titel, Jahr_Dok FROM Dokument WHERE PMID = '" + str(pmid_max) + "'")
    data = cur.fetchall()[0]
    titel_max = data[0]
    jahr_dok_max = data[1]
    mr.Markdown(f"#### Das Dokument \"{titel_max}\" (PMID: {pmid_max}) wurde seit {jahr_dok_max} insgesamt {anzahl_max} mal zitiert. Damit ist es das Dokument mit den meisten Zitaten in der Datenbank. ")
    # Durchschnitt emergente und korrigiert nicht-emergente Zitate pro Dokument
    cur.execute("SELECT count(PMID), sum(EK_Dok), sum(NEKK_Dok) FROM MeSH_Dok")
    data = cur.fetchall()[0]
    anzahl = data[0]
    ek_sum = data[1]
    nekk_sum = data[2]
    ek_schnitt = ek_sum / anzahl
    nekk_schnitt = nekk_sum / anzahl
    mr.Markdown(f"#### Auf Basis der analyisierten Dokumente wird ein Dokument durchschnittlich {ek_schnitt} mal emergent zitiert und {nekk_schnitt} mal nicht-emergent zitiert. (Betrachtung des korrigierten Zeitraums) ")

def aufnahmeJahrMeSH():
    cur.execute("SELECT Jahr_MeSH, count(Jahr_MeSH) FROM (SELECT Jahr_MeSH FROM MeSH ORDER BY Jahr_MeSH ASC) GROUP BY Jahr_MeSH")
    data = cur.fetchall()
    data_jahr = [x[0] for x in data]
    data_anzahl = [x[1] for x in data]
    jahr = []
    anzahl = []
    counter = 0
    for i in np.linspace(min(data_jahr), max(data_jahr), max(data_jahr)-min(data_jahr)+1, dtype = int):
        jahr.append(i)
        if i == data_jahr[counter]:
            anzahl.append(data_anzahl[counter])
            counter = counter + 1
        else:
            anzahl.append(0)
    fig = plt.figure(figsize=(15, 10))
    sub1 = plt.subplot(2, 2, 1)
    df1 = pd.DataFrame(anzahl,index=jahr)
    df1.plot.bar(ax = sub1, width=0.8, legend=None)
    sub1.set_title("Verteilung der Jahre, in denen die MeSH eingeführt wurden")
    sub1.set_xlabel("Jahr")
    sub1.set_ylabel("Anzahl eingeführter MeSH")
    


In [ ]:
show_code = mr.Checkbox(label="Quelltext ein-/ausblenden", value=True)
app = mr.App(title="Datenvisualisierung", 
             description="Visualisierung der Analyseergebnisse", show_code=show_code.value)
mr.Markdown(f"# Visualisierung von Zitationsdaten")

In [ ]:
alleMeSH_DB = DB_Select_Alle_MeSH()
mesh_auswahl_term = mr.Select(value=alleMeSH_DB[0], choices=alleMeSH_DB, label="Wählen Sie ein MeSH aus:")
mesh_auswahl_ui = DB_Select_MeSH_UI(mesh_auswahl_term.value)
# doks_titel_pmid_json, doks_titel = DB_Select_Alle_Dokumente_MeSH(mesh_auswahl_ui)
# dok_auswahl_titel = mr.Select(value=doks_titel[0], choices=doks_titel, label="Die in Verbindung stehenden Dokumente sind:")
# doks_pmid = doks_titel_pmid_json[dok_auswahl_titel.value]


In [ ]:
_ = infosAllgemein()
_ = visualisiereMeSHAnstieg()
_ = visualisiereKorrelationen()
_ = visualisiereGesamtUeberblick()
_ = visualisiereMeSHZeitstrahl(mesh_auswahl_ui)
_ = visualisiereRelativeVerlaufUndMeSH(mesh_auswahl_ui)
_ = aufnahmeJahrMeSH()
